## Importando as bibliotecas

In [1]:
import asyncio
import aiohttp
import nest_asyncio
import pandas as pd

## Estrutura do código
### Essa etapa já contém algumas modificações, como: 
### renomeação das colunas;
### remoção de colunas que não fazem sentido para o caso

In [2]:
# Permitir execução aninhada de loops de eventos
nest_asyncio.apply()

# Chave da API
api_key = '03142f95f9a513b7520bae4418120159'

# Definindo as listas de estados por região
norte = ["Rio Branco", "Macapá", "Manaus", "Belém", "Porto Velho", "Boa Vista", "Palmas"]
nordeste = ["Maceió", "Salvador", "Fortaleza", "São Luís", "João Pessoa", "Recife", "Natal", "Aracaju"]
centro_oeste = ["Brasília", "Goiânia", "Cuiabá", "Campo Grande"]
sudeste = ["Vitória", "Belo Horizonte", "Rio de Janeiro", "São Paulo"]
sul = ["Curitiba", "Porto Alegre", "Florianópolis"]

# Lista principal que contém todas as regiões
brasil = [norte, nordeste, centro_oeste, sudeste, sul]

In [3]:
# Função assíncrona para buscar dados do clima
async def fetch(session, cidade):
    url = f'https://api.openweathermap.org/data/2.5/forecast?q={cidade}&appid={api_key}&lang=pt_br&units=metric'
    async with session.get(url) as response:
        data = await response.json()
        cidade_id = data["city"]["id"]
        previsoes = []
        if response.status == 200:
            for item in data['list']:
                previsao = {
                    'cidade': cidade,
                    'data': item['dt_txt'],
                    'temperatura': int(item['main']['temp']),
                    'descricao': item['weather'][0]['description'],
                    'nuvens': item['clouds']['all'],
                    'dados_climaticos_id': cidade_id
                }
                previsoes.append(previsao)
        else:
            previsoes.append({'cidade': cidade, 'error': data.get('message', 'Erro desconhecido')})
        return previsoes

In [4]:
# Função assíncrona para buscar dados de todas as cidades
async def fetch_all(cidades):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, cidade) for cidade in cidades]
        responses = await asyncio.gather(*tasks)
        # Flatten the list of lists
        flattened_responses = [item for sublist in responses for item in sublist]
        return flattened_responses

In [5]:
# Flatten a lista principal para obter todas as cidades
todas_cidades = [cidade for regiao in brasil for cidade in regiao]

# Função para rodar o loop assíncrono
resultados = asyncio.run(fetch_all(todas_cidades))

In [6]:
# Converter resultados em DataFrame pandas
df = pd.DataFrame(resultados)
df

,cidade,data,temperatura,descricao,nuvens,dados_climaticos_id
0,Rio Branco,2024-06-02 00:00:00,26,algumas nuvens,13,3662574
1,Rio Branco,2024-06-02 03:00:00,24,céu limpo,7,3662574
2,Rio Branco,2024-06-02 06:00:00,21,céu limpo,0,3662574
3,Rio Branco,2024-06-02 09:00:00,20,céu limpo,0,3662574
4,Rio Branco,2024-06-02 12:00:00,22,céu limpo,3,3662574
...,...,...,...,...,...,...
1035,Florianópolis,2024-06-06 09:00:00,17,nublado,100,6323121
1036,Florianópolis,2024-06-06 12:00:00,19,nublado,76,6323121
1037,Florianópolis,2024-06-06 15:00:00,22,nublado,72,6323121
1038,Florianópolis,2024-06-06 18:00:00,22,nublado,82,6323121


## Salvando o resultado num .CSV

In [7]:
df.to_csv('previsao_dias.csv', index=False, encoding='utf-8')